In [18]:
# importing function from a different ipny 
import pandas as pd
import collections
from keras.initializers import Constant
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import operator
from itertools import product
import sys
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
#https://www.amazon.com/Neural-Networks-Deep-Learning-Textbook/dp/3319944622/ref=cm_cr_arp_d_product_top?ie=UTF8
#https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 20000

In [20]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[:10000], ham_index[:9997]), axis=0)
keras_data_new = keras_data.iloc[new_index]
labels = keras_data_new.isspam
keras_data_new.head()

,Subject,isspam
0,"Confidential :Soma:, Ci@lis, :P:ntermin",1
1,¢Â ÇªÁüÇÑ ¼³³¯ ÀÌº¥Æ® ÀÀ¸ðÇØ¼­ ºÎ¸ð´Ô²² ¼±¹°ÇÏ...,1
2,_na_,1
3,StOck 0ppurtunities - their sh0Oting,1
6,All your prescr[iption needs right here,1


In [21]:
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

In [22]:
embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

Found 400000 word vectors.
Processing text dataset


In [23]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_new.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]



Found 15425 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 2)


In [24]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')



Preparing embedding matrix.
Training model.


# LSTM Model

In [51]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[10000:19000], ham_index[9997:19000]), axis=0)
keras_data_test_set = keras_data.iloc[new_index]
labels_test_set = keras_data_test_set.isspam
keras_data_test_set.head()

,Subject,isspam
19223,RE: Daily Summary of Risk Data,0
19224,Softwares CDS all software under 15$ and 99$!,1
19226,Hotel Room Bargains at up to 70% off! Save in...,1
19227,RE: Greeley Gas Company,0
19228,"Any Software just in under 15-99$, Xp-adobe etc",1


In [52]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_test_set.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels_test_set = to_categorical(np.asarray(labels_test_set))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
#labels_test_set = labels_test_set[indices]
x_test = data

Found 15425 unique tokens.
Shape of data tensor: (17088, 1000)
Shape of label tensor: (19997, 2)


In [27]:
embeddings_LSTM = pd.read_csv('embeddings_LSTM_model.csv')

In [28]:
embeddings_LSTM_sort = embeddings_LSTM.sort_values(['val_loss'])
embeddings_LSTM_sort

,dl,fl1,fl2,fl3,kl,layer,optimizer,train_acc,train_loss,val_acc,val_loss
27,64,32,0,0,0,[1],Nadam,0.928179,0.180042,0.924231,0.182697
31,128,32,0,0,0,[1],Nadam,0.925866,0.178017,0.926732,0.188866
19,16,32,0,0,0,[1],Nadam,0.920928,0.190786,0.925981,0.194389
35,16,64,0,0,0,[1],Nadam,0.914677,0.205629,0.923231,0.196220
24,64,32,0,0,0,[1],RMSprop,0.911114,0.211151,0.919730,0.197624
7,32,16,0,0,0,[1],Nadam,0.920240,0.193405,0.920480,0.198082
11,64,16,0,0,0,[1],Nadam,0.922865,0.191926,0.923981,0.198908
15,128,16,0,0,0,[1],Nadam,0.919740,0.190584,0.923731,0.199320
29,128,32,0,0,0,[1],Adam,0.920428,0.194369,0.916729,0.200601
17,16,32,0,0,0,[1],Adam,0.912677,0.216945,0.923731,0.200625


In [29]:
def embedding_LSTM(fl1=16, fl2=16, fl3=16, dl=16, optimizer= 'RMSprop', kl = 5, layer =1): 
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = Bidirectional(LSTM(units = fl1, return_sequences=True))(embedded_sequences)
    x = GlobalMaxPool1D()(x)
    x = Dense(units=dl, activation="relu")(x)
    x = Dropout(0.1)(x)
    preds = Dense(2, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizer,
              metrics=['acc'])
    return model

## 1st model

In [32]:
kwargs = dict(fl1=32, fl2= 0, fl3=0, kl=5, dl=64, optimizer= ''.join('Nadam'), layer=1)
model = embedding_LSTM(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model1_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 919s 57ms/step - loss: 0.2846 - acc: 0.8809 - val_loss: 0.2135 - val_acc: 0.9162
Epoch 2/2
17088/17088 [==============================] - 205s 12ms/step


## 2nd model

In [33]:
kwargs = dict(fl1=32, fl2= 0, fl3=0, kl=5,dl=128, optimizer= ''.join('Nadam'), layer=1)
model = embedding_LSTM(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model2_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 884s 55ms/step - loss: 0.2822 - acc: 0.8769 - val_loss: 0.2185 - val_acc: 0.9110
Epoch 2/2
17088/17088 [==============================] - 195s 11ms/step


## 3rd model

In [36]:
kwargs = dict(fl1=32, fl2= 0, fl3=0, kl=5, dl=16, optimizer= ''.join('Nadam'), layer=1)
model = embedding_LSTM(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model3_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 922s 58ms/step - loss: 0.2972 - acc: 0.8745 - val_loss: 0.2527 - val_acc: 0.8945
Epoch 2/2
17088/17088 [==============================] - 197s 12ms/step


In [113]:
pred_test_y = 0.33*model1_test_y + 0.33*model2_test_y + 0.34*model3_test_y
A = pred_test_y[:,0]
B = pred_test_y[:,1]
pred_array =[]

for i in range(len(pred_test_y)):
    if (A[i]>0.99):
        pred_array.append(0)
    else:
        pred_array.append(1)

In [120]:
predictions = pred_array
actuals = labels_test_set.values
true_pos= 0
true_neg = 0
false_pos = 0
false_neg = 0
for i in range (17088):
    if ((predictions[i]==1) & (actuals[i]==1)):
        true_pos = true_pos+1
    elif((predictions[i]==0) & (actuals[i]==0)):
        true_neg = true_neg+1
    elif((predictions[i]==1) & (actuals[i]==0)):
        false_pos = false_pos +1
    elif((predictions[i]==0) & (actuals[i]==1)):
        false_neg = false_neg+1
prec=true_pos/(true_pos+false_pos)
recall = true_pos/(true_pos+false_neg)
accur=(true_pos+true_neg)/(true_pos+false_pos+ true_neg+ false_neg)
F1=2*(prec*recall/(prec+recall))
print('F1-score on test set is {0:.2f}%'.format(round(F1*100),2))

F1-score on test set is 64.00%
